In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from keras.optimizers import Adam

from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_absolute_error

In [2]:
params = {
    'test_size': 0.2,
    'random_state': 1337,
    'validation_split': 0.2
}

In [5]:
df = pd.read_parquet("../data/qsc_out.random_scan_nfp2.parquet")

In [6]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y0,y1,y2,y3,y4,y5,y6
0,0.133438,-0.092950,0.011193,-0.005795,0.000771,-0.000122,-0.713354,0.129141,1.482134,0.213466,0.241207,0.475203,0.852697,1.067530,0.342066
1,0.106896,0.107480,0.007474,0.007549,0.000577,0.000333,-0.718440,-0.309748,1.711563,0.178743,0.181471,0.403228,0.821898,1.052911,0.333320
2,0.100286,0.098256,0.006654,0.006411,0.000179,0.000225,-0.775836,0.064299,1.895026,0.154513,0.204983,0.335392,0.807669,0.965668,0.331046
3,0.120435,0.114909,0.009287,0.008526,0.000167,0.000237,-0.767415,0.016524,1.380982,0.196482,0.304152,0.364896,0.833857,1.286199,0.337564
4,-0.128050,0.160498,0.009165,-0.010457,-0.000347,0.000200,-0.719544,0.569769,0.962259,0.304666,0.529468,0.435512,0.834355,1.311465,0.340612


In [9]:
df.shape

(1000000, 15)

In [10]:
x_columns = [col for col in df.columns if col.startswith('x')]
y_columns = [col for col in df.columns if col.startswith('y')]

Y = df[y_columns].values
X = df[x_columns].values

In [11]:
def preprocess_data(X_train, X_test, Y_train, Y_test, params):
    scaler_x = StandardScaler().fit(X_train)
    scaler_y = StandardScaler().fit(Y_train)
    X_train = scaler_x.transform(X_train)
    X_test = scaler_x.transform(X_test)
    Y_train = scaler_y.transform(Y_train)
    Y_test = scaler_y.transform(Y_test)

    input_shape = X_train.shape[1]
    
    output_shape = Y_train.shape[1]
    return X_train, X_test, Y_train, Y_test, input_shape, output_shape, scaler_x, scaler_y

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=params['test_size'], 
                                                    random_state=params['random_state'])

X_train, X_test, Y_train, Y_test, input_shape, output_shape, scaler_x, scaler_y = preprocess_data(X_train, X_test, Y_train, Y_test, params)

In [12]:
X_train.mean(axis=0), X_train.std(axis=0) 

(array([-1.1327621e-08,  2.1925195e-09, -5.5531042e-09,  7.3113129e-09,
        -1.3621244e-09,  3.0783935e-09,  1.1375323e-08, -3.8778780e-09],
       dtype=float32),
 array([0.9993136 , 0.99944884, 0.99930733, 0.9995662 , 0.99942106,
        0.999486  , 0.9999612 , 0.9995944 ], dtype=float32))

## Appears to be drift, perhaps the sample is not big enough

In [13]:
X_test.mean(axis=0), X_test.std(axis=0) 

(array([-0.00667486,  0.00326206, -0.00185904, -0.00032165, -0.00151821,
        -0.0017868 , -0.00261114,  0.00337474], dtype=float32),
 array([0.99684393, 1.0012466 , 1.000381  , 1.002924  , 0.9978705 ,
        0.9981674 , 1.002929  , 1.0042428 ], dtype=float32))

In [14]:
Y_train.mean(axis=0), Y_train.std(axis=0) 

(array([-1.8581934e-09, -5.8630110e-09, -2.2679568e-10,  1.4387071e-10,
         4.0571391e-09,  5.9454890e-09, -1.5963987e-09], dtype=float32),
 array([1.0002788 , 0.9998443 , 0.9994234 , 0.9997594 , 0.99986285,
        0.99981064, 0.9998727 ], dtype=float32))

In [15]:
Y_test.mean(axis=0), Y_test.std(axis=0)

(array([-0.00227338,  0.00395122,  0.00310262,  0.00321028,  0.00143382,
         0.00332455, -0.00154998], dtype=float32),
 array([0.99452734, 1.0021232 , 1.0060604 , 1.008807  , 0.9999831 ,
        1.0016111 , 0.99785244], dtype=float32))

## Dummy regressor

In [16]:
from sklearn.dummy import DummyRegressor

In [17]:
regr = MultiOutputRegressor(DummyRegressor(strategy="mean")).fit(X_train, Y_train)
regr.predict(X_train)

array([[ 5.0783155e-10, -6.2942505e-09,  6.4849853e-10, ...,
        -2.5177003e-09, -1.7833710e-09,  3.0517577e-09],
       [ 5.0783155e-10, -6.2942505e-09,  6.4849853e-10, ...,
        -2.5177003e-09, -1.7833710e-09,  3.0517577e-09],
       [ 5.0783155e-10, -6.2942505e-09,  6.4849853e-10, ...,
        -2.5177003e-09, -1.7833710e-09,  3.0517577e-09],
       ...,
       [ 5.0783155e-10, -6.2942505e-09,  6.4849853e-10, ...,
        -2.5177003e-09, -1.7833710e-09,  3.0517577e-09],
       [ 5.0783155e-10, -6.2942505e-09,  6.4849853e-10, ...,
        -2.5177003e-09, -1.7833710e-09,  3.0517577e-09],
       [ 5.0783155e-10, -6.2942505e-09,  6.4849853e-10, ...,
        -2.5177003e-09, -1.7833710e-09,  3.0517577e-09]], dtype=float32)

In [18]:
mean_absolute_error(Y_train, regr.predict(X_train))

0.63178873

In [19]:
mean_absolute_error(Y_test, regr.predict(X_test))

0.6326293

## Train a linear regression for debugging

In [20]:
regr = MultiOutputRegressor(Ridge(random_state=params['random_state'])).fit(X_train, Y_train)
regr.predict(X_train)

array([[-6.92282617e-02, -1.54025435e-01,  3.67904827e-02, ...,
         8.91883895e-02,  1.03378534e-01, -4.63521540e-01],
       [-2.72226613e-02, -6.77829310e-02, -2.33512698e-03, ...,
         5.69161810e-02,  1.49837554e-01, -8.12186658e-01],
       [-1.34016767e-01, -2.40030453e-01,  7.71415457e-02, ...,
         1.01409532e-01,  5.49193025e-02, -7.21290754e-03],
       ...,
       [ 2.92817801e-01,  5.27068496e-01, -1.71872094e-01, ...,
        -2.38644585e-01, -1.26464874e-01,  2.43872833e-02],
       [ 8.15712567e-03,  1.36674335e-02,  7.52523541e-04, ...,
        -1.09247095e-03, -7.78605090e-03,  9.96023323e-03],
       [-8.90059397e-02, -1.57348022e-01,  3.95330749e-02, ...,
         8.14322233e-02,  4.80963849e-02, -4.81136180e-02]], dtype=float32)

In [21]:
mean_absolute_error(Y_train, regr.predict(X_train))

0.61463964

In [22]:
mean_absolute_error(Y_test, regr.predict(X_test))

0.6148032

## Simplest neural network

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [24]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(input_shape, activation="relu", name="layer_in"),
        layers.Dense(128, activation="relu", name="layer2"),
        layers.Dense(64, activation="relu", name="layer3"),
        layers.Dense(output_shape, name="layer_out"),
    ]
)

model(X_train[0:1])

<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
array([[ 0.0311353 ,  0.06876908, -0.1295784 , -0.09838126, -0.34702867,
         0.04909335, -0.15615612]], dtype=float32)>

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_in (Dense)            (1, 8)                    72        
                                                                 
 layer2 (Dense)              (1, 128)                  1152      
                                                                 
 layer3 (Dense)              (1, 64)                   8256      
                                                                 
 layer_out (Dense)           (1, 7)                    455       
                                                                 
Total params: 9935 (38.81 KB)
Trainable params: 9935 (38.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanAbsoluteError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanAbsoluteError()],
)

In [27]:
print("Fit model on training data")
history = model.fit(
    X_train,
    Y_train,
    batch_size=64,
    epochs=400,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(X_test, Y_test),
)

Fit model on training data
Epoch 1/400
12500/12500 [==============================] - 6s 470us/step - loss: 0.2756 - mean_absolute_error: 0.2756 - val_loss: 0.2315 - val_mean_absolute_error: 0.2315
Epoch 2/400
12500/12500 [==============================] - 6s 459us/step - loss: 0.2196 - mean_absolute_error: 0.2196 - val_loss: 0.2118 - val_mean_absolute_error: 0.2118
Epoch 3/400
12500/12500 [==============================] - 6s 479us/step - loss: 0.2045 - mean_absolute_error: 0.2045 - val_loss: 0.2013 - val_mean_absolute_error: 0.2013
Epoch 4/400
12500/12500 [==============================] - 6s 455us/step - loss: 0.1946 - mean_absolute_error: 0.1946 - val_loss: 0.1896 - val_mean_absolute_error: 0.1896
Epoch 5/400
12500/12500 [==============================] - 6s 458us/step - loss: 0.1882 - mean_absolute_error: 0.1882 - val_loss: 0.1882 - val_mean_absolute_error: 0.1882
Epoch 6/400
12500/12500 [==============================] - 6s 456us/step - loss: 0.1836 - mean_absolute_error: 0.1836 

Epoch 96/400
12500/12500 [==============================] - 6s 485us/step - loss: 0.1540 - mean_absolute_error: 0.1540 - val_loss: 0.1578 - val_mean_absolute_error: 0.1578
Epoch 97/400
12500/12500 [==============================] - 6s 451us/step - loss: 0.1539 - mean_absolute_error: 0.1539 - val_loss: 0.1541 - val_mean_absolute_error: 0.1541
Epoch 98/400
12500/12500 [==============================] - 6s 451us/step - loss: 0.1539 - mean_absolute_error: 0.1539 - val_loss: 0.1547 - val_mean_absolute_error: 0.1547
Epoch 99/400
12500/12500 [==============================] - 6s 464us/step - loss: 0.1539 - mean_absolute_error: 0.1539 - val_loss: 0.1564 - val_mean_absolute_error: 0.1564
Epoch 100/400
12500/12500 [==============================] - 6s 469us/step - loss: 0.1539 - mean_absolute_error: 0.1539 - val_loss: 0.1558 - val_mean_absolute_error: 0.1558
Epoch 101/400
12500/12500 [==============================] - 6s 453us/step - loss: 0.1538 - mean_absolute_error: 0.1538 - val_loss: 0.1539 

12500/12500 [==============================] - 6s 453us/step - loss: 0.1515 - mean_absolute_error: 0.1515 - val_loss: 0.1581 - val_mean_absolute_error: 0.1581
Epoch 191/400
12500/12500 [==============================] - 6s 454us/step - loss: 0.1515 - mean_absolute_error: 0.1515 - val_loss: 0.1517 - val_mean_absolute_error: 0.1517
Epoch 192/400
12500/12500 [==============================] - 6s 449us/step - loss: 0.1515 - mean_absolute_error: 0.1515 - val_loss: 0.1533 - val_mean_absolute_error: 0.1533
Epoch 193/400
12500/12500 [==============================] - 6s 473us/step - loss: 0.1515 - mean_absolute_error: 0.1515 - val_loss: 0.1523 - val_mean_absolute_error: 0.1523
Epoch 194/400
12500/12500 [==============================] - 7s 596us/step - loss: 0.1516 - mean_absolute_error: 0.1516 - val_loss: 0.1562 - val_mean_absolute_error: 0.1562
Epoch 195/400
12500/12500 [==============================] - 8s 648us/step - loss: 0.1516 - mean_absolute_error: 0.1516 - val_loss: 0.1534 - val_mean

12500/12500 [==============================] - 6s 450us/step - loss: 0.1509 - mean_absolute_error: 0.1509 - val_loss: 0.1538 - val_mean_absolute_error: 0.1538
Epoch 285/400
12500/12500 [==============================] - 6s 464us/step - loss: 0.1509 - mean_absolute_error: 0.1509 - val_loss: 0.1514 - val_mean_absolute_error: 0.1514
Epoch 286/400
12500/12500 [==============================] - 5s 433us/step - loss: 0.1509 - mean_absolute_error: 0.1509 - val_loss: 0.1547 - val_mean_absolute_error: 0.1547
Epoch 287/400
12500/12500 [==============================] - 5s 431us/step - loss: 0.1510 - mean_absolute_error: 0.1510 - val_loss: 0.1538 - val_mean_absolute_error: 0.1538
Epoch 288/400
12500/12500 [==============================] - 6s 442us/step - loss: 0.1509 - mean_absolute_error: 0.1509 - val_loss: 0.1497 - val_mean_absolute_error: 0.1497
Epoch 289/400
12500/12500 [==============================] - 6s 456us/step - loss: 0.1509 - mean_absolute_error: 0.1509 - val_loss: 0.1517 - val_mean

12500/12500 [==============================] - 6s 460us/step - loss: 0.1503 - mean_absolute_error: 0.1503 - val_loss: 0.1523 - val_mean_absolute_error: 0.1523
Epoch 379/400
12500/12500 [==============================] - 6s 459us/step - loss: 0.1503 - mean_absolute_error: 0.1503 - val_loss: 0.1529 - val_mean_absolute_error: 0.1529
Epoch 380/400
12500/12500 [==============================] - 6s 464us/step - loss: 0.1502 - mean_absolute_error: 0.1502 - val_loss: 0.1525 - val_mean_absolute_error: 0.1525
Epoch 381/400
12500/12500 [==============================] - 6s 460us/step - loss: 0.1503 - mean_absolute_error: 0.1503 - val_loss: 0.1528 - val_mean_absolute_error: 0.1528
Epoch 382/400
12500/12500 [==============================] - 6s 463us/step - loss: 0.1502 - mean_absolute_error: 0.1502 - val_loss: 0.1530 - val_mean_absolute_error: 0.1530
Epoch 383/400
12500/12500 [==============================] - 6s 459us/step - loss: 0.1502 - mean_absolute_error: 0.1502 - val_loss: 0.1521 - val_mean

In [26]:
mean_absolute_error(Y_train, model.predict(X_train))

742/742 [==============================] - 0s 315us/step


0.10905128717603396

In [27]:
mean_absolute_error(Y_test, model.predict(X_test))

186/186 [==============================] - 0s 311us/step


0.11408455206590504